In [1]:
import yaml
import ast
import snowflake.snowpark.functions as F
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session
from snowflake.ml.registry import Registry

In [3]:
def check_and_update(df, model_name):
    if df.empty:
        return "V_1"
    elif df[df["name"] == model_name].empty:
        return "V_1"
    else:
        # Increment model_version if df is not a pandas Series
        lst = sorted(ast.literal_eval(df["versions"][0]))
        last_value = lst[-1]
        prefix, num = last_value.rsplit("_", 1)
        new_last_value = f"{prefix}_{int(num)+1}"
        lst[-1] = new_last_value
        return new_last_value

In [4]:
session = Session.builder.configs(SnowflakeLoginOptions()).getOrCreate()

reg = Registry(session=session)
reg_df = reg.show_models()
model_version = check_and_update(reg_df, "TITANIC")

with open("../params.yaml") as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

cfg["model"]["version"] = model_version

with open("../params.yaml", "w") as f:
    cfg = yaml.dump(cfg, stream=f, default_flow_style=False, sort_keys=False)